# initialisation

## truc

In [1]:
'''
Import des librairies nécéssaires pour faire tourner le code
---------------------
librairies externes
---------------------
si ça ne tourne pas, c'est qu'il faut installer le package en tapant dans la console (Ctrl+ù sur VScode):

conda install package

'''
import pandas as pd
import geopandas as gpd
import os
from datetime import datetime
# from time import sleep
# from openpyxl import Workbook
# from openpyxl import load_workbook
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium.plugins import Draw


'''
Libraires internes
---------------------
Si ça ne tourne pas il faut ajouter au début de votre code:

import sys
sys.path.append(r'chemin d'accès au dossier contenant la bibliothèque interne')
---------------------
Exemple:
> from library.PY_lib.coordinates import *

le chemin d'accès du dossier "library" est D:\Programmes\E.nova\python\PyEnova\flight_track\library

il faut donc ajouter au dessus :

import sys
sys.path.append(r'D:\Programmes\E.nova\python\PyEnova\flight_track')

L'opération n'est en principe a faire qu'une seule fois, tant que vos bibliothèques sont rangées au même endroit, 
d'où l'importance de ne pas éparpiller les dossiers de librairies un peu partout
'''
# import sys
# sys.path.append(r'D:\Programmes\E.nova\python\PyEnova\flight_track')

from PY_lib.data_retriever import *


## Inputs
je rajoute du texte

In [2]:
#Save folders
input_folder = r'Data\input'
output_folder = r'Data\output'

In [75]:
from pyproj import CRS, Transformer
from shapely.geometry import Point, Polygon
from shapely.ops import transform
from geopandas.datasets import get_path
#Zone definition

# Zone = 'circle'
Zone = 'polygon'
# Zone = 'region'

Coordinates = ((0,40), (0,45), (5,45),(5,40))
'''longitude, latitude in WGS 84'''
Radius = 1000.0         
'''radius in meters'''
Region = 'Québec'       #marche pas encore
'''Region, country, city name, it may not be in database'''

'Region, country, city name, it may not be in database'

In [76]:
# provinces = gpd.read_file(r'Data\Geo_Data\provinces\ne_50m_admin_1_states_provinces.shp')
# display(provinces[['name','name_fr','sov_a3','adm1_code', 'geometry']])

# Country = gpd.read_file(r'Data\Geo_Data\countries\ne_110m_admin_0_countries.shp')
# display(Country[['NAME','NAME_FR','ISO_A3','FORMAL_EN','geometry']])

# City = gpd.read_file(r'Data\Geo_Data\city\ne_50m_populated_places.shp')
# display(City[['NAME','NAME_FR','SOV_A3','ADM1NAME','geometry']])

# # a=pd.DataFrame(world)
# # world.loc[world['name']==Region]
# # world.loc[world['name']==Region, 'geometry']

In [77]:
match Zone :
    case 'circle':
        # Azimuthal equidistant projection
        aeqd_proj = CRS.from_proj4(f"+proj=aeqd +lat_0={Coordinates[1]} +lon_0={Coordinates[0]} +x_0=0 +y_0=0")
        tfmr = Transformer.from_proj(aeqd_proj, aeqd_proj.geodetic_crs)
        buf = Point(Coordinates).buffer(Radius)  # distance in miles
        area = gpd.GeoSeries(Polygon(transform(tfmr.transform, buf).exterior.coords[:]), crs='EPSG:4326')

    case 'polygon':
        area = gpd.GeoSeries(Polygon(Coordinates), crs='EPSG:4326')

    case 'region' :
        world = gpd.read_file(r'Data\Geo_Data\ne_110m_admin_0_countries.shp')
        poly=world.loc[world['name'] == Region, 'geometry']
        area= gpd.GeoSeries(poly, crs='EPSG:4326')
        
    case _:
        raise SystemExit('undefined area')
    
area.explore()

In [78]:
'''
Paramètres de génération de données:
'''
n = 1
'''automatisation (0 pour oui, 1 pour non)'''
r = 1
'''request left'''
t = 3600 
'''temps entre chaque requete (en secondes)'''
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
'''date d'extraction des données'''

'''
Paramètres pour délimiter la zone
'''
choice = 1
'''1=quadrilatère, 2 = circulaire'''
taille_zone = 100
'''en km si choix 2'''

Data_generation = False 
'''Définir True pour faire une requete API lors de l'exectution de ce notebook,
cette option est surtout utile en mode démo et/ou pour le developpeur'''

"Définir True pour faire une requete API lors de l'exectution de ce notebook,\ncette option est surtout utile en mode démo et/ou pour le developpeur"

In [79]:
if Data_generation:
    get_data(n, r - 1, t)

## Filtre

In [80]:
'''
Fonction qui filtre les données en fonction de la zone à étudier.
Pour changer la taille de la zone, il faut modifier les fichiers round_coordinates.txt ou square_coordinates.txt 
'''
def onefiledata(filename, taille_zone, choice, r_latitudes, r_longitude, s_latitude, s_longitude):
    # print(filename)
    data = pd.read_excel(filename)
    data = pd.DataFrame(data)

    if choice == 1:
        t_lat = s_latitude
        t_lng = s_longitude
        # filtered_data = []+
        for index, row in data.iterrows():
            lat = row['lat']
            lng = row['lng']
            if sort_elements(t_lat[0], t_lat[1], t_lat[2], t_lat[3], lat, t_lng[0], t_lng[1], t_lng[2], t_lng[3], lng) == 0:
                data.loc[index,'distance'] = True
        filtered_data=data.where(data['distance']==True)

    if choice == 2:
        for index, row in data.iterrows():
            lat = row['lat']
            lng = row['lng']
            for ref_lat, ref_lng in zip(r_latitudes, r_longitude):
                data.loc[index,'distance'] = calculate_distance(lat, lng, float(ref_lat), float(ref_lng))
        filtered_data=data.where(data['distance']<=taille_zone)
    # display(filtered_data.dropna())
    return filtered_data.dropna()

# Post-traitement

In [81]:
'''
Fonction qui charge les données dans le notebook pour postraitement 
et les sauvegardent dans un excel dans le dossier output
'''
round_path = r'round_coordinates.txt'
square_path = r'square_coordinates.txt'

r_latitudes, r_longitude = read_coordinates(round_path)
s_latitude, s_longitude = read_coordinates(square_path)

filename = os.listdir(input_folder)

df=[]
for idf, fname in enumerate(filename):
    path2 = os.path.join(input_folder,fname)
    filtered_data=onefiledata(path2,taille_zone, choice, r_latitudes, r_longitude, s_latitude, s_longitude)
    
    # prefix = 'excel_'
    prefix = ''
    suffix = '_flight'
    date_str=''
    idx1 = fname.index(prefix)
    idx2 = fname.index(suffix)
    for idx in range(idx1 +len(prefix)+1,idx2):
        date_str=date_str+fname[idx]
    filtered_data['Date']=date_str
    
    df.append(filtered_data)

if choice==1:
    new_filename = os.path.join(output_folder,f'{current_time}_square_sorted_.xlsx')
if choice==2: 
    new_filename = os.path.join(output_folder,f'{current_time}_round_sorted_.xlsx')

filtered_df = pd.concat([*df]) 

filtered_df.to_excel(new_filename, index=False)

print(f"Tout a été enregistré dans le fichier : {new_filename}")

C:\Users\arthu\AppData\Local\Temp\ipykernel_11956\2613398457.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.loc[index,'distance'] = True
C:\Users\arthu\AppData\Local\Temp\ipykernel_11956\2613398457.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.loc[index,'distance'] = True
C:\Users\arthu\AppData\Local\Temp\ipykernel_11956\2613398457.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.loc[index,'distance'] = True


Tout a été enregistré dans le fichier : Data\output\2024-02-02_17-39-49_square_sorted_.xlsx


In [82]:
'''
On a ici filtered_df qui est notre dataFrame de base avec toutes nos données, les calculs sont faits, il ne reste que le postraitement.

Pour couvrir les différents cas sur la carte, on a besoin de différencier les avions qui n'apparaissent qu'une seule fois de ceux qui apparaissent plusieurs fois, 
car on ne les traite pas de la même manière:

Avions uniques : position sur la carte, flèche orientée selon leur cap
Avions qui ont plusieurs occurences: Pareil sauf que les occurences sont reliées entre elles

J'ai donc créé 2 plus petits dataframes:
Avions uniques : df_d
'''
idx=filtered_df.set_index(['hex']).index.drop_duplicates(keep=False)  
'''on créée une liste d'index (ici la colone hex), et on supprime toutes les lignes qui ont plus d'une occurence'''
df_d=filtered_df.set_index(['hex']).loc[idx].sort_index()
'''on filtre le dataframe de base avec seulement les index uniques'''
# df_d=df_d.set_index(['hex']).sort_index()
'''On définit les colones hex et date comme index, pour la lisibilité'''

df_p=filtered_df.set_index(['hex']).drop(index=idx)
'''on supprime les lignes avec seulement les index uniques, 
donc on ne garde que les lignes avec des index en double, triple, etc'''
df_p=df_p.reset_index().set_index(['hex', 'Date']).sort_index()
'''On définit les colones hex et date comme index, pour la lisibilité,
et c'est là qu'avoir du multiindex prend tout son sens.'''



"On définit les colones hex et date comme index, pour la lisibilité,\net c'est là qu'avoir du multiindex prend tout son sens."

In [83]:
'''
Affichage des deux dataframes:
'''
# display(df_d)
# display(df_p)

'\nAffichage des deux dataframes:\n'

# OUTPUTS

## map folium

In [84]:
'''
Carte avec les avions affichés, après pas mal d'essais sur plotly, je suis finalement passé sur folium
J'ai aussi reçu un coup de main de chatGPT, dont j'ai tout de même du adapter le code pour que ça marche, 
j'ai laissé ses commentaires.
'''
# Create a Folium Map
m = folium.Map(location=[df_p['lat'].mean(), df_p['lng'].mean()], zoom_start=6)

# Iterate through your data and create lines and markers
for i in df_p.index.get_level_values(0):
    locations = []  # Accumulate coordinates for each i    
    for j in df_p.loc[i].index.get_level_values(0):
        angle = df_p.loc[(i,j), 'dir']  # Angle in degrees
        # Create a marker
        marker = folium.Marker(
            location=(df_p.loc[(i,j), 'lat'], df_p.loc[(i,j), 'lng']),
            icon=folium.DivIcon(
                html=f'<div><div style="transform: rotate({angle}deg); border: 8px solid transparent; border-bottom-color: blue; width: 0; height: 0;"></div></div>'
            ),
            tooltip=i
        )

        m.add_child(marker)

        # Accumulate coordinates for the line
        locations.append((df_p.loc[(i,j), 'lat'], df_p.loc[(i,j), 'lng']))
    
    # Create a line (PolyLine)
    line = folium.PolyLine(
        locations=locations,
        color='blue',
        weight=2,
        opacity=1,
        tooltip=i
    )
    
    m.add_child(line)

for i in df_d.index:
    angle = df_d.loc[i, 'dir']  # Angle in degrees
    # Create a marker
    marker = folium.Marker(
        location=(df_d.loc[i, 'lat'], df_d.loc[i, 'lng']),
        icon=folium.DivIcon(
            html=f'<div><div style="transform: rotate({angle}deg); border: 8px solid transparent; border-bottom-color: blue; width: 0; height: 0;"></div></div>'
        ),
        tooltip=i
    )

    m.add_child(marker)


# Add the drawing tools for interactive features (optional)
draw = Draw(export=True)
draw.add_to(m)

# Display the map

# m.save(r'C:\Users\e.NOVA Aerospace\Documents\PYTHON\flight_track\Data\output\folium_map.html')
m.save(os.path.join(output_folder,f'folium_map.html'))
m


In [85]:
'''
Tentative 2 avec plotly que je laisse pour la beauté de la chose, mais j'arrivais pas à afficher une flèche orientée selon le cap
'''
fig = go.Figure()

for i in df_p.index.get_level_values(0):
    fig.add_trace(go.Scattermapbox(
        mode="lines+markers", #
        lon=df_p.loc[i, 'lng'],
        lat=df_p.loc[i, 'lat'],
        name=i, 
        marker=dict(
            symbol="arrow",
            size=10,
            angle=df_p.loc[i, 'dir'],
            ),
        ))
    


fig.update_layout(
    margin={'l': 113, 't': 24, 'b': 22, 'r': 115},
    mapbox=dict(style='carto-positron',
                center=go.layout.mapbox.Center(lon=7, lat=43.5),
                pitch=0,
                zoom=6)
)

fig.show()

In [86]:
'''
tentative 1 avec plotly
rapide a coder, mais peu d'options disponibles
'''
fig=px.scatter_geo(
    data_frame=filtered_df,
    lat='lat',
    lon='lng',
    hover_name='reg_number',
    color='flag',
    hover_data=['alt', 'speed'],
    )
fig.update_layout(width=1500, height=900)
fig.write_html(os.path.join('output',f'{current_time}_Flight_map'))
fig


FileNotFoundError: [Errno 2] No such file or directory: 'output\\2024-02-02_17-39-49_Flight_map'

In [ ]:
'''
Exemple de graph très rapidement sortable avec plotly
On peut cliquer sur la légende à droite pour afficher/cacher des objets
'''
fig = px.scatter(
    data_frame=filtered_df,
    y='alt',
    x='speed',
    color='Date'
)
fig.show()

## Nombre avion par date

In [ ]:
'''
comptage du nombre d'avions par date
'''
dff=pd.DataFrame(index=filtered_df['Date'].unique(), columns=['Nombre'])
for i in filtered_df['Date'].unique():
    dff.loc[i, 'Nombre']=filtered_df['Date'].where(filtered_df['Date']==i).count()

'''
affichage avec px.
'''
fig = px.bar(
    data_frame=dff,
    y='Nombre',
    x=dff.index,
    # color='Date'
)
fig.show()

# fig.write_html(r'C:\Users\e.NOVA Aerospace\Documents\PYTHON\flight_track\output\nb_avion.html')
fig.write_html(os.path.join(output_folder,f'nb_avion.html'))

In [ ]:
filtered_df.columns

Index(['hex', 'reg_number', 'flag', 'lat', 'lng', 'alt', 'dir', 'speed',
       'v_speed', 'squawk', 'flight_number', 'flight_icao', 'flight_iata',
       'dep_icao', 'dep_iata', 'arr_icao', 'arr_iata', 'airline_icao',
       'airline_iata', 'aircraft_icao', 'updated', 'status', 'Date',
       'distance'],
      dtype='object')

In [ ]:
df_airline=pd.DataFrame(index=filtered_df['arr_iata'].unique(), columns=['Nombre'])
for i in filtered_df['arr_iata'].unique():
    df_airline.loc[i, 'Nombre']=filtered_df['arr_iata'].where(filtered_df['arr_iata']==i).count()

'''
Exemple de graph très rapidement sortable avec plotly
On peut cliquer sur la légende à droite pour afficher/cacher des objets
'''
fig = px.bar(
    data_frame=df_airline,
    y='Nombre',
    x=df_airline.index,
    # color=df_airline.index
)
fig.show()

In [ ]:
df_aircraft=pd.DataFrame(index=filtered_df['aircraft_icao'].unique(), columns=['Nombre'])
for i in filtered_df['aircraft_icao'].unique():
    df_aircraft.loc[i, 'Nombre']=filtered_df['aircraft_icao'].where(filtered_df['aircraft_icao']==i).count()
    df_aircraft=df_aircraft.sort_index()
'''
Exemple de graph très rapidement sortable avec plotly
On peut cliquer sur la légende à droite pour afficher/cacher des objets
'''
fig = px.bar(
    data_frame=df_aircraft,
    y='Nombre',
    x=df_aircraft.index,
    title="type d'avion"

)
fig.show()